# Работа с файлами

In [18]:
file = open('data/text_w.txt','w')

file.write('Hello \n')
file.write('World')

file.close()

In [29]:
file = open('data/text_w.txt','r')

print(file.read())

file.close()

Hello 
World


In [31]:
file = open('data/text_w.txt','r')

print(file.read(4))
print(file.read())

file.close()

Hell
o 
World


In [34]:
file = open('data/text_w.txt','r')

for line in file:
    print(line, end = '')

file.close()

Hello 
World

In [9]:
file = open('data/text_a.txt','a')

file.write('Hello World\n')

file.close()

При работе с файлами удобно пользоваться менеджером «With ... as».
При вызове этой конструкции, внутри блока выполняются действия внутри файла, при этом после выполнения блока файл автоматически закрывается, поэтому строку ```file.close()``` прописывать не нужно

In [10]:
data = input('Введите текст: ')

with open('data/text_input.txt','a') as file:
    file.write(data+'\n')